In [ ]:
# Perform our typical data science and PyMC imports

# Import cytoolz for data manipulation
import cytoolz.curried as ctc

# Import PyMC and supporting packages
import arviz as az
import pymc as pm
import preliz as pz

# Import other "data science" packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Generate synthetic data
N_samples = [30, 30, 30] # number of samples
G_samples = [18, 3, 3] # number of good samples
group_idx = np.repeat(np.arange(len(N_samples)), N_samples)

# Perhaps `cytoolz` might simplify this code
data = []
for i in range(0, len(N_samples)):
    data.extend(np.repeat([1, 0],
                          [G_samples[i], N_samples[i] - G_samples[i]]))

In [ ]:
# Let's write the model in PyMC.
with pm.Model() as model_h:
    # hyperpriors
    mu = pm.Beta('mu', 1, 1)
    nu = pm.HalfNormal('nu', 10)

    # prior
    theta = pm.Beta('theta', mu=mu, nu=nu, shape=len(N_samples))

    # likelihood
    y = pm.Bernoulli('y', p = theta[group_idx], observed=data)

    idata_h = pm.sample(random_seed=4591)

In [ ]:
# Let's write the same model **without** using a hierarchical model
with pm.Model() as model_nh:
    # Prior
    theta = pm.Beta('theta', alpha=1, beta=1, shape=len(N_samples))

    # Likelihood
    y = pm.Bernoulli('y', p=theta[group_idx], observed=data)

    idata_nh = pm.sample(random_seed=4591)

In [ ]:
az.plot_trace(idata_h)
az.plot_trace(idata_nh)
plt.show()

In [ ]:
az.summary(idata_h)

In [ ]:
az.summary(idata_nh)

In [ ]:
az.plot_posterior(idata_h)
az.plot_posterior(idata_nh)
plt.show()

In [ ]:
axes = az.plot_forest(
    [idata_h, idata_nh],
    model_names=['hierarchical', 'non-hierarchical'],
    var_names='theta',
    combined=True,
    r_hat=False,
    ess=False,
    figsize=(12, 3),
    colors='cycle',
)
y_lims = axes[0].get_ylim()
plt.show()

I do not notice any significant difference between the two models.
Both models capture the difference between first group and the
last two groups. Additionally, both models capture the similarity
between the last two groups.

However, as the author points out, in the first group, the means
of the two groups are "pulled apart"; that is, using the
hierarchical model, the mean of the first group is "pulled toward"
the mean of the other two groups. Without a hierarchical model, the
mean is unaffected by the other two groups.

In [ ]:
# Load the tips data set
tips = pd.read_csv('../data/tips.csv')

In [ ]:
# We'll preprocess the tips data set
categories = np.array(['Thur', 'Fri', 'Sat', 'Sun'])
tip = tips['tip'].values
idx = pd.Categorical(tips['day'], categories=categories).codes
coords = {'days': categories, 'days_flat': categories[idx]}

In [ ]:
# We now perform the same modeling we did in chapter 2
with pm.Model(coords=coords) as comparing_groups_nh:
    mu = pm.HalfNormal('mu', sigma=5, dims='days')
    sigma = pm.HalfNormal('sigma', sigma=1, dims='days')
    y = pm.Gamma('y', mu=mu[idx], sigma=sigma[idx],
                 observed=tip, dims='days_flat')

    idata_cg_nh = pm.sample(random_seed=4591)

In [ ]:
# Let's now create a similar model with partial pooling across the
# days of the week.
with pm.Model(coords=coords) as comparing_groups_h:
    # Hyper-prior
    hp_mu = pm.HalfNormal('hp_mu', sigma=5, dims='days')

    # Prior
    mu = pm.Normal('mu', mu=hp_mu, sigma=5, dims='days')
    sigma = pm.HalfNormal('sigma', sigma=1, dims='days')

    # Likelihood
    y = pm.Gamma('y', mu=mu[idx], sigma=sigma[idx],
                 observed=tip, dims='days_flat')

    idata_cg_h = pm.sample(random_seed=4591)

In [ ]:
az.plot_trace(idata_cg_nh)
az.plot_trace(idata_cg_h)
plt.show()

In [ ]:
az.summary(idata_cg_nh)

In [ ]:
az.summary(idata_cg_h)

In [ ]:
az.plot_posterior(idata_cg_nh)
az.plot_posterior(idata_cg_h)
plt.show()

In [ ]:
axes = az.plot_forest(
    [idata_cg_nh, idata_cg_h],
    model_names=['non-hierarchical', 'hierarchical'],
    var_names='mu',
    combined=True,
    r_hat=False,
    ess=False,
    figsize=(12, 3),
    colors='cycle',
)
y_lims = axes[0].get_ylim()

The author chose a different set of models. I think I understand
the reason (but I'm not fully certain).

In [ ]:
# The first model just sets a hyper-prior over `mu`
with pm.Model(coords=coords) as comparing_groups_h_00:
    # Hyper-prior
    # I'm uncertain about the reason the author chose a Gamma distribution.
    # From the AI Overview generated by Google, these reasons seem
    # to be pertinent.
    # Positive support: The Gamma distribution only generates
    # **positive** values which seems suitable for tips.
    # I'm uncertain about the other reasons supplied.
    mu_g = pm.Gamma('mu_g', mu=5, sigma=2)

    # Prior
    mu = pm.HalfNormal('mu', sigma=mu_g, dims='days')
    sigma = pm.HalfNormal('sigma', sigma=1, dims='days')

    # Likelihood
    y = pm.Gamma('y', mu=mu[idx], sigma=sigma[idx],
                 observed=tip, dims='days_flat')

    # Magic inference button
    idata_cg_h_00 = pm.sample(random_seed=4591)

In [ ]:
# Here's the second (alternative) model
with pm.Model(coords=coords) as comparing_groups_h_01:
    # Hyper-priors
    mu_g = pm.Gamma('mu_g', mu=5, sigma=2)
    sigma_g = pm.Gamma('sigma_g', mu=2, sigma=1.5)

    # Priors
    mu = pm.Gamma('mu', mu=mu_g, sigma=sigma_g, dims='days')
    sigma = pm.HalfNormal('sigma', sigma=1, dims='days')

    # Likelihood
    y = pm.Gamma('y', mu=mu[idx], sigma=sigma[idx],
                 observed=tip, dims='days_flat')

    # The magic inference button
    idata_cg_h_01 = pm.sample(random_seed=4591)

In [ ]:
# The author also suggests coding up a non-centered version. These
# kinds of models are discussed in chapter 4.
with pm.Model(coords=coords) as comparing_groups_h_02:
    # Hyper-priors
    mu_g = pm.Gamma('mu_g', mu=5, sigma=2)
    sigma_g = pm.HalfNormal('sigma_g', sigma=2)

    # Priors
    mu_g_offset = pm.Normal('mu_g_offset', sigma=2, dims='days')
    mu = pm.Deterministic('mu', mu_g + mu_g_offset, dims='days')
    sigma = pm.HalfNormal('sigma', sigma=1, dims='days')

    # Likelihood
    y = pm.Gamma('y', mu=mu[idx], sigma=sigma[idx],
                 observed=tip, dims='days_flat')

    # Again, the magical inference button
    idata_cg_h_02 = pm.sample(random_seed=4591, target_accepted=0.99)

In [ ]:
axes = az.plot_forest(
    [idata_cg_nh, idata_cg_h_00, idata_cg_h_01, idata_cg_h_02],
    model_names=['non-hierarchical', 'hierarchical_00',
                 'hierarchical_01', 'hierarchical_02'],
    var_names='mu',
    combined=True,
    r_hat=False,
    figsize=(12, 3),
    colors='cycle',
)
y_lims = axes[0].get_ylim()
axes[0].vlines(idata_cg_h_00.posterior['mu_g'].mean(),
               *y_lims,
               colors='k',
               ls=":")
plt.show()

As the author points out, "We can see little difference between
the hierarchical and non-hierarchical models when we use a
HalfNormal distribution for the mean, `mu`, (`hierarchical_00`
and `non-hierarchical`, respectively). The difference is
larger when we use a Gamma distribution (`hierarchical_01`).

Interestingly, using the non-centered version (`hierarchical_02`)
again seems to reduce the difference.